In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

# โหลดค่าจากไฟล์ .env
load_dotenv()

# เชื่อมต่อกับฐานข้อมูล SQLite
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# สร้างตาราง products
cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price REAL NOT NULL,
    category TEXT
)
''')

# สร้างตาราง orders 
cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    quantity INTEGER,
    order_date TEXT,
    FOREIGN KEY (product_id) REFERENCES products (product_id)
)
''')

# ยืนยันการเปลี่ยนแปลง
conn.commit()


In [ ]:
# สร้าง Faker object สำหรับข้อมูลจำลอง
from faker import Faker
fake = Faker()

# สร้างข้อมูลสินค้า
product_data = {
    'product_id': range(1, 101),  # สร้าง 100 สินค้า
    'name': [fake.word() + ' ' + fake.word() for _ in range(100)],
    'price': np.random.uniform(10, 1000, 100).round(2),
    'category': np.random.choice(['อิเล็กทรอนิกส์', 'เสื้อผ้า', 'อาหาร', 'เครื่องสำอาง', 'เฟอร์นิเจอร์'], 100)
}

# สร้าง DataFrame สำหรับสินค้า
df_products = pd.DataFrame(product_data)

# สร้างข้อมูลคำสั่งซื้อ
num_orders = 1000  # สร้าง 1000 คำสั่งซื้อ
order_data = {
    'order_id': range(1, num_orders + 1),
    'product_id': np.random.choice(df_products['product_id'], num_orders),
    'quantity': np.random.randint(1, 10, num_orders),
    'order_date': [fake.date_between(start_date='-1y', end_date='today') for _ in range(num_orders)]
}

# สร้าง DataFrame สำหรับคำสั่งซื้อ
df_orders = pd.DataFrame(order_data)

# บันทึกข้อมูลลงในฐานข้อมูล SQLite
df_products.to_sql('products', conn, if_exists='replace', index=False)
df_orders.to_sql('orders', conn, if_exists='replace', index=False)

# ยืนยันการเปลี่ยนแปลง
conn.commit()

print("จำนวนสินค้าที่เพิ่มเข้าไป:", len(df_products))
print("จำนวนคำสั่งซื้อที่เพิ่มเข้าไป:", len(df_orders))
